In [ ]:
pip install mercantile

In [ ]:
pip install pillow

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144628 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.23-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
!mkdir -p ndrive
!google-drive-ocamlfuse ndrive

In [ ]:
!ls /content/ndrive/TruckDataset_20_zoom

0.0.png     13.218.png	18.173.png  2.240.png	27.236.png  4.76.png
0.100.png   13.219.png	18.174.png  22.40.png	27.237.png  4.77.png
0.101.png   1.321.png	18.175.png  2.241.png	27.238.png  4.78.png
0.102.png   13.21.png	18.176.png  22.41.png	27.239.png  4.79.png
0.103.png   13.220.png	18.177.png  2.242.png	27.23.png   4.7.png
0.104.png   13.221.png	18.178.png  22.42.png	27.240.png  4.80.png
0.105.png   13.222.png	18.179.png  2.243.png	27.241.png  4.81.png
0.106.png   13.223.png	18.17.png   22.43.png	27.242.png  4.82.png
0.107.png   13.224.png	18.180.png  2.244.png	27.243.png  4.83.png
0.108.png   13.225.png	18.181.png  22.44.png	27.244.png  4.84.png
0.109.png   13.226.png	18.182.png  2.245.png	27.245.png  4.85.png
0.10.png    13.227.png	18.183.png  22.45.png	27.246.png  4.86.png
0.110.png   13.228.png	18.184.png  2.246.png	27.247.png  4.87.png
0.111.png   13.229.png	18.185.png  22.46.png	27.248.png  4.88.png
0.112.png   1.322.png	18.186.png  2.247.png	27.249.png  4.89.png
0.113.png   13

# Converting From Lat/Lng to Tilesets

In [ ]:
#Dallas location
lat_lng = [32.7767, -96.7970]
# lat_lng = [43.640918, -79.371478]

In [ ]:
delta=0.05
tl = [lat_lng[0]+delta, lat_lng[1]-delta]
br = [lat_lng[0]-delta, lat_lng[1]+delta]
z = 20 # Set the resolution (max at 15)

# Pulling the Images

In [ ]:
import mercantile
tl_tiles = mercantile.tile(tl[1],tl[0],z)
br_tiles = mercantile.tile(br[1],br[0],z)
x_tile_range =[tl_tiles.x,br_tiles.x];print(x_tile_range)
y_tile_range = [tl_tiles.y,br_tiles.y];print(y_tile_range)

[242200, 242491]
[422967, 423313]


In [ ]:
import requests # The requests package allows use to call URLS
import shutil   # shutil will be used to copy the image to the local
# Loop over the tile ranges
for i,x in enumerate(range(x_tile_range[0],x_tile_range[1]+1)):
  for j,y in enumerate(range(y_tile_range[0],y_tile_range[1]+1)):
    r =requests.get('https://api.mapbox.com/v4/mapbox.satellite/'+ str(z)+'/'+str(x)+'/'+str(y)+'@2x.pngraw?access_token=pk.eyJ1IjoibWluaGRuZ3V5ZW4iLCJhIjoiY2tneHlmaHUwMG92cDJwcGlyajJoMGtsdyJ9.cJ01130Y1oU-wD-kcvzuAw', stream=True) 
    with open('/content/ndrive/TruckDataset_20_zoom/' + str(i) + '.' + str(j) + '.png','wb') as f:
       r.raw.decode_content = True
       shutil.copyfileobj(r.raw, f)

Composing the Final Map

In [ ]:
# Import the image, math and os libraries
import PIL
import math 
from os import listdir
from os.path import isfile, join
# Loop over the elevation and satellite image set
# for img_name in ['satellite']:
   # Make a list of the image names   
image_files = ['/content/ndrive/TruckDataset_20_zoom/' + f for f in listdir('/content/ndrive/TruckDataset_20_zoom')]
    # Open the image set using pillow
images = [PIL.Image.open(x) for x in image_files]
   # Calculate the number of image tiles in each direction
edge_length_x = x_tile_range[1] - x_tile_range[0]
edge_length_y = y_tile_range[1] - y_tile_range[0]
edge_length_x = max(1,edge_length_x)
edge_length_y = max(1,edge_length_y)
   # Find the final composed image dimensions  
width, height = images[0].size
total_width = width*edge_length_x
total_height = height*edge_length_y
   # Create a new blank image we will fill in
composite = PIL.Image.new('RGB', (total_width, total_height))
   # Loop over the x and y ranges
y_offset = 0
for i in range(0,edge_length_x):
  x_offset = 0
  for j in range(0,edge_length_y):
        # Open up the image file and paste it into the composed image at the given offset position
    tmp_img = PIL.Image.open('/content/ndrive/TruckDataset_20_zoom/' + str(i) + '.' + str(j) + '.png')
    composite.paste(tmp_img, (y_offset,x_offset))
    x_offset += width # Update the width
    y_offset += height # Update the height
# Save the final image
composite.save('/content/ndrive/TruckDataset_20_zoom/'+'final'+'.png')